### Faiss
Facebook AI Similarity search(Faiss) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning.

In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

In [2]:
loader = TextLoader("speech.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000,chunk_overlap=30)
docs = text_splitter.split_documents(documents)


In [3]:
docs

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
 Document(metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct our

In [4]:
embeddings = OllamaEmbeddings(model ="deepseek-r1")
db = FAISS.from_documents(docs, embeddings)

C:\Users\PCC\AppData\Local\Temp\ipykernel_11548\620486272.py:1: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model ="deepseek-r1")


In [5]:
db

In [6]:
### Querying the database
query = "How does the speaker describe the desired outcome of the war?"
docs = db.similarity_search(query)
docs

[Document(id='eec3951f-4e69-43a7-8b2d-d848ea61783e', metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
 Document(id='7fcc10fd-196f-429c-b4a6-a54cbd073f8f', metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. 

In [7]:
docs[0].page_content

'…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'

### As a Retriever 
We can also convert the vectorstore into a Retriever class. This allows us to easily use it in other LangChain methods, which largely work with retrievers.

In [9]:
retriever = db.as_retriever()
docs = retriever.invoke(query)

In [10]:
docs[0].page_content

'…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'

### Similarity Search with score
There are some FAISS specific methods. One of them is similarity_search_with_score, which allows you to return not only the document but also the distance score of the query to them. The returned distance score is L2 distance. Therefore, a lower score is better.

In [11]:
docs_and_score = db.similarity_search_with_score(query)
docs_and_score

[(Document(id='eec3951f-4e69-43a7-8b2d-d848ea61783e', metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
  np.float32(11649.91)),
 (Document(id='7fcc10fd-196f-429c-b4a6-a54cbd073f8f', metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundat

In [12]:
embedding_vector = embeddings.embed_query(query)
embedding_vector

[-0.31087222695350647,
 -0.37334388494491577,
 -1.9563056230545044,
 0.8515955209732056,
 0.17088662087917328,
 -3.25812029838562,
 1.7774566411972046,
 0.7688387632369995,
 -1.4315460920333862,
 0.1852962076663971,
 -1.105049729347229,
 0.9117358922958374,
 3.062713623046875,
 -0.11575986444950104,
 0.5966061949729919,
 1.5692214965820312,
 -0.314670205116272,
 0.8488945364952087,
 -1.267452597618103,
 -2.0320801734924316,
 -0.1879916936159134,
 0.6132000684738159,
 -1.1971486806869507,
 -1.6093754768371582,
 -0.6262486577033997,
 -1.3374437093734741,
 -0.013283845968544483,
 -1.50553297996521,
 -1.272672176361084,
 2.147721767425537,
 1.7957392930984497,
 -1.0887724161148071,
 1.1823437213897705,
 0.7470608353614807,
 -0.9721699357032776,
 -1.4209321737289429,
 0.31973710656166077,
 0.00041528523433953524,
 -1.210858941078186,
 0.8349984884262085,
 -1.3838584423065186,
 -0.11434613913297653,
 0.5696833729743958,
 0.28830185532569885,
 -1.3484121561050415,
 -0.3411180078983307,
 -2.22

In [13]:
docs_score = db.similarity_search_by_vector(embedding_vector)
docs_score

[Document(id='eec3951f-4e69-43a7-8b2d-d848ea61783e', metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
 Document(id='7fcc10fd-196f-429c-b4a6-a54cbd073f8f', metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. 

In [14]:
## Saving And Loading
db.save_local("faiss_index")

In [17]:
new_db = FAISS.load_local("faiss_index",embeddings,allow_dangerous_deserialization=True)

In [19]:
docs = new_db.similarity_search(query)

In [20]:
docs

[Document(id='eec3951f-4e69-43a7-8b2d-d848ea61783e', metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
 Document(id='7fcc10fd-196f-429c-b4a6-a54cbd073f8f', metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. 